# LoanShark AI - ML Pipeline
## Predatory Loan Detection with Hybrid Scoring

This notebook contains:
1. Feature Extraction (regex-based)
2. Data Loading & Preprocessing
3. Model Training (Logistic Regression + XGBoost)
4. Model Evaluation & Selection
5. Inference Functions
6. Hybrid Scoring Logic
7. Explanations & Highlights Generation
8. End-to-End Testing

## 1. Setup & Imports

In [16]:
import os
import re
import json
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

# Try to import XGBoost, fallback to RandomForest if not available
try:
    import xgboost as xgb
    USE_XGBOOST = True
    print("✓ XGBoost available")
except ImportError:
    USE_XGBOOST = False
    print("⚠ XGBoost not available, will use RandomForest instead")

# Set random seed for reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("✓ All imports successful")

✓ XGBoost available
✓ All imports successful


## 2. Feature Extraction Functions

Extract 30+ structured features from loan contract text using regex and keyword matching.

In [17]:
def extract_apr(text):
    """Extract APR value from text."""
    patterns = [
        r'APR[:\s]+([0-9]+\.?[0-9]*)%',
        r'Annual Percentage Rate[:\s]+([0-9]+\.?[0-9]*)%',
        r'interest rate[:\s]+([0-9]+\.?[0-9]*)%',
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return float(match.group(1))
    return -1  # Not found

def extract_fee(text, fee_type):
    """Extract specific fee value."""
    patterns = [
        rf'{fee_type}[:\s]+\$([0-9]+\.?[0-9]*)',
        rf'{fee_type}[:\s]+([0-9]+\.?[0-9]*)%',
    ]
    for pattern in patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return float(match.group(1))
    return -1

def extract_term_days(text):
    """Extract loan term in days."""
    # Look for "Term: X days" or "X months"
    day_match = re.search(r'Term[:\s]+([0-9]+)\s*days?', text, re.IGNORECASE)
    if day_match:
        return int(day_match.group(1))
    
    month_match = re.search(r'Term[:\s]+([0-9]+)\s*months?', text, re.IGNORECASE)
    if month_match:
        return int(month_match.group(1)) * 30  # Convert to days
    
    return -1

def count_keywords(text, keywords):
    """Count occurrences of keywords (case-insensitive)."""
    count = 0
    text_lower = text.lower()
    for keyword in keywords:
        count += text_lower.count(keyword.lower())
    return count

def has_pattern(text, patterns):
    """Check if any pattern exists in text."""
    for pattern in patterns:
        if re.search(pattern, text, re.IGNORECASE):
            return 1
    return 0

def extract_features(text):
    """Extract all features from loan contract text."""
    features = {}
    
    # === APR & Cost Features ===
    apr = extract_apr(text)
    features['apr_value'] = apr if apr > 0 else 0
    features['apr_missing'] = 1 if apr == -1 else 0
    features['apr_over_100'] = 1 if apr > 100 else 0
    features['apr_over_300'] = 1 if apr > 300 else 0
    
    # === Fee Features ===
    features['late_fee_value'] = max(0, extract_fee(text, 'Late Fee'))
    features['origination_fee_value'] = max(0, extract_fee(text, 'Origination Fee'))
    features['service_fee_value'] = max(0, extract_fee(text, 'Service Fee'))
    features['renewal_fee_value'] = max(0, extract_fee(text, 'Renewal Fee'))
    
    fee_keywords = ['fee', 'charge', 'penalty', 'service fee', 'processing']
    features['fee_word_count'] = count_keywords(text, fee_keywords)
    
    # Fee per $100 pattern (very predatory)
    features['mentions_per_100'] = has_pattern(text, [
        r'\$[0-9]+\s*per\s*\$100',
        r'per\s*\$100\s*borrowed'
    ])
    
    # === Term & Payment Features ===
    term = extract_term_days(text)
    features['term_days'] = term if term > 0 else 0
    features['term_very_short'] = 1 if 0 < term <= 14 else 0
    
    # Payment schedule
    features['has_single_payment_due'] = has_pattern(text, [
        r'single payment',
        r'due on payday',
        r'payment due.*payday'
    ])
    
    features['has_monthly_payment'] = has_pattern(text, [
        r'monthly',
        r'payment schedule.*monthly'
    ])
    
    # === Clause Detection (Predatory Patterns) ===
    features['has_rollover_or_renewal'] = has_pattern(text, [
        r'rollover',
        r'renew',
        r'renewal',
        r'extend',
        r'automatically renew'
    ])
    
    features['has_balloon_payment'] = has_pattern(text, [
        r'balloon payment',
        r'balloon'
    ])
    
    features['has_auto_debit'] = has_pattern(text, [
        r'auto.*debit',
        r'automatic.*debit',
        r'auto.*pay'
    ])
    
    features['has_continuous_debit'] = has_pattern(text, [
        r'repeatedly debit',
        r'continuous.*authorization',
        r'debit.*repeatedly',
        r'until paid'
    ])
    
    features['has_wage_assignment'] = has_pattern(text, [
        r'wage assignment',
        r'paycheck.*assignment'
    ])
    
    features['has_arbitration'] = has_pattern(text, [
        r'arbitration',
        r'binding arbitration'
    ])
    
    features['has_class_action_waiver'] = has_pattern(text, [
        r'class action waiver',
        r'waive.*class action',
        r'no class action'
    ])
    
    features['has_jury_waiver'] = has_pattern(text, [
        r'jury.*waiver',
        r'waive.*jury',
        r'no jury trial'
    ])
    
    features['has_confession_of_judgment'] = has_pattern(text, [
        r'confession of judgment',
        r'confess.*judgment'
    ])
    
    features['has_employer_contact'] = has_pattern(text, [
        r'contact.*employer',
        r'employer.*collection'
    ])
    
    # === Transparency Features (Positive Signals) ===
    features['has_clear_disclosure'] = has_pattern(text, [
        r'APR.*disclosed',
        r'fee schedule.*included',
        r'clearly.*disclosed'
    ])
    
    features['has_transparency_language'] = has_pattern(text, [
        r'transparency',
        r'disclosure',
        r'right to sue',
        r'may revoke',
        r'can cancel'
    ])
    
    # Fee ambiguity (negative signal)
    features['has_fee_ambiguity'] = has_pattern(text, [
        r'fees may apply',
        r'may change without notice',
        r'see external schedule',
        r'additional fees'
    ])
    
    # === Document Statistics ===
    features['doc_length_words'] = len(text.split())
    features['num_money_amounts'] = len(re.findall(r'\$[0-9,]+', text))
    features['num_percentages'] = len(re.findall(r'[0-9]+\.?[0-9]*%', text))
    
    # === Risk Ratios ===
    if apr > 0 and term > 0:
        features['apr_to_term_ratio'] = apr / term  # High APR + short term = extreme risk
    else:
        features['apr_to_term_ratio'] = 0
    
    return features

# Test feature extraction
test_text = """PAYDAY LOAN AGREEMENT
Loan Amount: $300
APR: 420%
Service Fee: $25 per $100 borrowed
Term: 14 days
Binding arbitration required."""

test_features = extract_features(test_text)
print("✓ Feature extraction working")
print(f"Sample features: APR={test_features['apr_value']}, Term={test_features['term_days']}, Arbitration={test_features['has_arbitration']}")

✓ Feature extraction working
Sample features: APR=420.0, Term=14, Arbitration=1


## 3. Data Loading & Preprocessing

In [18]:
def load_dataset(dataset_path='dataset'):
    """Load all loan documents and extract features."""
    data = []
    labels = []
    
    # Load safe loans (label = 0)
    safe_dir = Path(dataset_path) / 'safe'
    for file_path in safe_dir.glob('*.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            features = extract_features(text)
            data.append(features)
            labels.append(0)
    
    # Load predatory loans (label = 1)
    predatory_dir = Path(dataset_path) / 'predatory'
    for file_path in predatory_dir.glob('*.txt'):
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            features = extract_features(text)
            data.append(features)
            labels.append(1)
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    df['label'] = labels
    
    return df

# Load dataset
print("Loading dataset...")
df = load_dataset()
print(f"✓ Loaded {len(df)} samples")
print(f"  - Safe loans: {(df['label']==0).sum()}")
print(f"  - Predatory loans: {(df['label']==1).sum()}")
print(f"  - Features: {len(df.columns)-1}")

# Display sample
print("\nSample features:")
df.head()

Loading dataset...
✓ Loaded 70 samples
  - Safe loans: 38
  - Predatory loans: 32
  - Features: 31

Sample features:


,apr_value,apr_missing,apr_over_100,apr_over_300,late_fee_value,origination_fee_value,service_fee_value,renewal_fee_value,fee_word_count,mentions_per_100,...,has_confession_of_judgment,has_employer_contact,has_clear_disclosure,has_transparency_language,has_fee_ambiguity,doc_length_words,num_money_amounts,num_percentages,apr_to_term_ratio,label
0,0.0,1,0,0,15.0,1.0,0.0,0.0,6,0,...,0,0,0,0,0,101,2,2,0.000000,0
1,18.0,0,0,0,10.0,10.0,0.0,0.0,7,0,...,0,0,1,1,0,71,3,1,0.100000,0
2,0.0,1,0,0,0.0,2.0,0.0,0.0,4,0,...,0,0,0,0,0,65,2,2,0.000000,0
3,30.0,0,0,0,20.0,0.0,0.0,0.0,5,0,...,0,0,0,0,0,65,3,1,0.111111,0
4,12.0,0,0,0,0.0,0.0,0.0,0.0,4,0,...,0,0,0,0,0,64,2,2,0.025000,0


In [19]:
# Prepare train/test split
X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y
)

print(f"✓ Train/test split complete")
print(f"  - Training samples: {len(X_train)}")
print(f"  - Test samples: {len(X_test)}")

# Save feature names for later use
feature_names = list(X.columns)
print(f"\n✓ Feature names saved ({len(feature_names)} features)")

✓ Train/test split complete
  - Training samples: 56
  - Test samples: 14

✓ Feature names saved (31 features)


## 4. Model Training

In [20]:
# Train Logistic Regression
print("Training Logistic Regression...")
lr_model = LogisticRegression(random_state=RANDOM_STATE, max_iter=1000)
lr_model.fit(X_train, y_train)
print("✓ Logistic Regression trained")

# Train XGBoost or RandomForest
if USE_XGBOOST:
    print("\nTraining XGBoost...")
    tree_model = xgb.XGBClassifier(
        random_state=RANDOM_STATE,
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1
    )
    tree_model.fit(X_train, y_train)
    print("✓ XGBoost trained")
    tree_name = "XGBoost"
else:
    print("\nTraining RandomForest...")
    tree_model = RandomForestClassifier(
        random_state=RANDOM_STATE,
        n_estimators=100,
        max_depth=10
    )
    tree_model.fit(X_train, y_train)
    print("✓ RandomForest trained")
    tree_name = "RandomForest"

Training Logistic Regression...


✓ Logistic Regression trained

Training XGBoost...
✓ XGBoost trained


## 5. Model Evaluation & Selection

In [21]:
def evaluate_model(model, X_test, y_test, model_name):
    """Evaluate model and print metrics."""
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision_safe = precision_score(y_test, y_pred, pos_label=0)
    precision_pred = precision_score(y_test, y_pred, pos_label=1)
    recall_safe = recall_score(y_test, y_pred, pos_label=0)
    recall_pred = recall_score(y_test, y_pred, pos_label=1)
    f1_safe = f1_score(y_test, y_pred, pos_label=0)
    f1_pred = f1_score(y_test, y_pred, pos_label=1)
    
    print(f"\n{'='*50}")
    print(f"{model_name} Evaluation")
    print(f"{'='*50}")
    print(f"Accuracy: {accuracy:.3f}")
    print(f"\nSafe Loans (Class 0):")
    print(f"  Precision: {precision_safe:.3f}")
    print(f"  Recall: {recall_safe:.3f}")
    print(f"  F1-Score: {f1_safe:.3f}")
    print(f"\nPredatory Loans (Class 1):")
    print(f"  Precision: {precision_pred:.3f}")
    print(f"  Recall: {recall_pred:.3f}")
    print(f"  F1-Score: {f1_pred:.3f}")
    
    print(f"\nConfusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print(f"  TN={cm[0,0]}, FP={cm[0,1]}")
    print(f"  FN={cm[1,0]}, TP={cm[1,1]}")
    
    return {
        'accuracy': accuracy,
        'f1_predatory': f1_pred,
        'precision_safe': precision_safe,
        'precision_predatory': precision_pred,
        'recall_predatory': recall_pred
    }

# Evaluate both models
lr_metrics = evaluate_model(lr_model, X_test, y_test, "Logistic Regression")
tree_metrics = evaluate_model(tree_model, X_test, y_test, tree_name)


Logistic Regression Evaluation
Accuracy: 0.929

Safe Loans (Class 0):
  Precision: 0.889
  Recall: 1.000
  F1-Score: 0.941

Predatory Loans (Class 1):
  Precision: 1.000
  Recall: 0.833
  F1-Score: 0.909

Confusion Matrix:
[[8 0]
 [1 5]]
  TN=8, FP=0
  FN=1, TP=5

XGBoost Evaluation
Accuracy: 0.929

Safe Loans (Class 0):
  Precision: 0.889
  Recall: 1.000
  F1-Score: 0.941

Predatory Loans (Class 1):
  Precision: 1.000
  Recall: 0.833
  F1-Score: 0.909

Confusion Matrix:
[[8 0]
 [1 5]]
  TN=8, FP=0
  FN=1, TP=5


In [22]:
# Select best model
print("\n" + "="*50)
print("MODEL SELECTION")
print("="*50)

# Selection criteria: Best F1 on predatory class AND precision on safe class > 0.75
if (tree_metrics['f1_predatory'] > lr_metrics['f1_predatory'] and 
    tree_metrics['precision_safe'] > 0.75):
    chosen_model = tree_model
    chosen_name = tree_name
    chosen_metrics = tree_metrics
    print(f"✓ Selected: {tree_name}")
else:
    chosen_model = lr_model
    chosen_name = "Logistic Regression"
    chosen_metrics = lr_metrics
    print(f"✓ Selected: Logistic Regression")

print(f"  F1-Score (Predatory): {chosen_metrics['f1_predatory']:.3f}")
print(f"  Precision (Safe): {chosen_metrics['precision_safe']:.3f}")
print(f"  Recall (Predatory): {chosen_metrics['recall_predatory']:.3f}")


MODEL SELECTION
✓ Selected: Logistic Regression
  F1-Score (Predatory): 0.909
  Precision (Safe): 0.889
  Recall (Predatory): 0.833


## 6. Save Model & Schema

In [23]:
# Create models directory
os.makedirs('models', exist_ok=True)

# Save model
model_path = 'models/loanshark_model.joblib'
joblib.dump(chosen_model, model_path)
print(f"✓ Model saved to {model_path}")

# Save feature schema
schema = {
    'feature_names': feature_names,
    'model_type': chosen_name,
    'metrics': chosen_metrics,
    'trained_on_samples': len(X_train)
}

schema_path = 'models/feature_schema.json'
with open(schema_path, 'w') as f:
    json.dump(schema, f, indent=2)
print(f"✓ Feature schema saved to {schema_path}")

# Save training report
report_path = 'models/training_report.txt'
with open(report_path, 'w') as f:
    f.write("LoanShark AI - Training Report\n")
    f.write("="*50 + "\n\n")
    f.write(f"Model: {chosen_name}\n")
    f.write(f"Training Samples: {len(X_train)}\n")
    f.write(f"Test Samples: {len(X_test)}\n")
    f.write(f"Features: {len(feature_names)}\n\n")
    f.write(f"Accuracy: {chosen_metrics['accuracy']:.3f}\n")
    f.write(f"F1-Score (Predatory): {chosen_metrics['f1_predatory']:.3f}\n")
    f.write(f"Precision (Safe): {chosen_metrics['precision_safe']:.3f}\n")
    f.write(f"Recall (Predatory): {chosen_metrics['recall_predatory']:.3f}\n")
print(f"✓ Training report saved to {report_path}")

✓ Model saved to models/loanshark_model.joblib
✓ Feature schema saved to models/feature_schema.json
✓ Training report saved to models/training_report.txt


## 7. Inference Functions

In [24]:
# Load model and schema
def load_model_and_schema():
    """Load trained model and feature schema."""
    try:
        model = joblib.load('models/loanshark_model.joblib')
        with open('models/feature_schema.json', 'r') as f:
            schema = json.load(f)
        return model, schema
    except Exception as e:
        print(f"⚠ Error loading model: {e}")
        return None, None

def predict_ml(text, model=None, schema=None):
    """Get ML prediction for loan text."""
    if model is None or schema is None:
        model, schema = load_model_and_schema()
    
    if model is None:
        return None  # Model not available
    
    # Extract features
    features_dict = extract_features(text)
    
    # Convert to feature vector in correct order
    feature_vector = [features_dict.get(name, 0) for name in schema['feature_names']]
    feature_vector = np.array(feature_vector).reshape(1, -1)
    
    # Get probability
    try:
        prob = model.predict_proba(feature_vector)[0][1]  # Probability of predatory
        ml_score = round(prob * 100)
        return {
            'ml_prob': prob,
            'ml_score': ml_score,
            'features': features_dict
        }
    except Exception as e:
        print(f"⚠ Prediction error: {e}")
        return None

print("✓ Inference functions defined")

✓ Inference functions defined


## 8. Hybrid Scoring Logic

In [25]:
def calculate_rule_score(features):
    """Calculate rule-based score (0-100)."""
    score = 0
    
    # APR scoring
    apr = features.get('apr_value', 0)
    if apr > 400:
        score += 40
    elif apr > 300:
        score += 35
    elif apr > 100:
        score += 25
    elif apr > 50:
        score += 15
    elif apr > 36:
        score += 10
    
    # Clause scoring (each worth points)
    if features.get('has_arbitration', 0):
        score += 10
    if features.get('has_class_action_waiver', 0):
        score += 10
    if features.get('has_rollover_or_renewal', 0):
        score += 12
    if features.get('has_continuous_debit', 0):
        score += 8
    if features.get('mentions_per_100', 0):
        score += 10
    if features.get('term_very_short', 0):
        score += 8
    if features.get('has_employer_contact', 0):
        score += 7
    
    # Reduce score for positive signals
    if features.get('has_clear_disclosure', 0):
        score -= 10
    if features.get('has_transparency_language', 0):
        score -= 5
    if features.get('has_monthly_payment', 0):
        score -= 5
    
    return max(0, min(100, score))  # Clamp to 0-100

def calculate_confidence(features):
    """Calculate confidence score based on extraction quality."""
    confidence_score = 100
    
    if features.get('apr_missing', 0):
        confidence_score -= 20
    if features.get('term_days', 0) == 0:
        confidence_score -= 10
    if features.get('fee_word_count', 0) < 2:
        confidence_score -= 15
    if features.get('doc_length_words', 0) < 50:
        confidence_score -= 20
    
    if confidence_score >= 80:
        return "High"
    elif confidence_score >= 50:
        return "Medium"
    else:
        return "Low"

def hybrid_score(text, ml_result=None):
    """Calculate final hybrid score combining rules + ML."""
    features = extract_features(text)
    rule_score = calculate_rule_score(features)
    confidence = calculate_confidence(features)
    
    # Get ML score if available
    if ml_result is None:
        ml_result = predict_ml(text)
    
    if ml_result is None:
        # ML not available, use rules only
        final_score = rule_score
        ml_score = None
        ml_prob = None
    else:
        ml_score = ml_result['ml_score']
        ml_prob = ml_result['ml_prob']
        
        # Adaptive weighting based on extraction quality
        if features.get('apr_missing', 0) and features.get('fee_word_count', 0) < 3:
            # Low confidence in features, trust rules more
            rule_weight = 0.9
            ml_weight = 0.1
        elif confidence == "High":
            # High confidence, balance rules and ML
            rule_weight = 0.5
            ml_weight = 0.5
        else:
            # Medium confidence, default weights
            rule_weight = 0.6
            ml_weight = 0.4
        
        final_score = round(rule_weight * rule_score + ml_weight * ml_score)
    
    # Apply hard floor rules (prevent ML from smoothing obvious predatory signals)
    apr = features.get('apr_value', 0)
    if apr > 400:
        final_score = max(final_score, 85)  # Force Predatory
    if (features.get('has_arbitration', 0) and 
        features.get('has_class_action_waiver', 0) and 
        apr > 100):
        final_score = max(final_score, 75)  # Force High Risk minimum
    
    # Map to label
    if final_score <= 20:
        label = "Safe"
    elif final_score <= 50:
        label = "Caution"
    elif final_score <= 80:
        label = "High Risk"
    else:
        label = "Predatory"
    
    return {
        'score': final_score,
        'label': label,
        'confidence': confidence,
        'rule_score': rule_score,
        'ml_score': ml_score,
        'ml_prob': ml_prob,
        'features': features
    }

print("✓ Hybrid scoring functions defined")

✓ Hybrid scoring functions defined


## 9. Explanations & Highlights Generation

In [26]:
def generate_reasons(features):
    """Generate priority-based reasons for the risk score."""
    reasons = []
    
    apr = features.get('apr_value', 0)
    
    # Priority 1: Extreme APR
    if apr > 300:
        reasons.append(f"APR is {apr:.0f}%, which is extremely high and predatory.")
    elif apr > 100:
        reasons.append(f"APR is {apr:.0f}%, significantly above typical rates (36% is considered high).")
    elif apr > 50:
        reasons.append(f"APR is {apr:.0f}%, which is elevated compared to standard loans.")
    
    # Priority 2: Debt cycle mechanisms
    if features.get('has_rollover_or_renewal', 0):
        reasons.append("Loan includes rollover/renewal clauses that can trap borrowers in debt cycles.")
    
    # Priority 3: Legal traps
    if features.get('has_arbitration', 0) and features.get('has_class_action_waiver', 0):
        reasons.append("Mandatory arbitration + class action waiver severely limits your legal rights.")
    elif features.get('has_arbitration', 0):
        reasons.append("Mandatory arbitration clause found (you waive your right to sue in court).")
    
    # Priority 4: Payment access
    if features.get('has_continuous_debit', 0):
        reasons.append("Lender can repeatedly debit your account, risking overdraft fees and loss of control.")
    elif features.get('has_auto_debit', 0):
        reasons.append("Automatic debit authorization may make it difficult to manage payments.")
    
    # Priority 5: Fee structure
    if features.get('mentions_per_100', 0):
        reasons.append("Fees charged per $100 borrowed compound quickly on short-term loans.")
    
    # Priority 6: Short term
    if features.get('term_very_short', 0):
        reasons.append("Very short repayment term (14 days or less) makes it difficult to repay without rolling over.")
    
    # Priority 7: Employer contact
    if features.get('has_employer_contact', 0):
        reasons.append("Lender may contact your employer for collection, risking your job.")
    
    # Positive signals (if score is low)
    if features.get('has_clear_disclosure', 0) and apr < 50:
        reasons.append("Loan includes clear APR and fee disclosures.")
    
    if features.get('has_transparency_language', 0) and apr < 50:
        reasons.append("Contract includes borrower-friendly language and rights.")
    
    # Return top 5 reasons
    return reasons[:5]

def extract_highlights(text, features):
    """Extract highlighted snippets from the contract."""
    highlights = []
    
    # Find APR mention
    apr_match = re.search(r'(APR[:\s]+[0-9]+\.?[0-9]*%)', text, re.IGNORECASE)
    if apr_match and features.get('apr_value', 0) > 100:
        highlights.append({
            "text": apr_match.group(1),
            "category": "ExcessiveCost"
        })
    
    # Find per $100 fee pattern
    per100_match = re.search(r'(\$[0-9]+\s*per\s*\$100[^.]{0,30})', text, re.IGNORECASE)
    if per100_match:
        highlights.append({
            "text": per100_match.group(1),
            "category": "ExcessiveCost"
        })
    
    # Find arbitration clause
    arb_match = re.search(r'(binding arbitration[^.]{0,50})', text, re.IGNORECASE)
    if arb_match:
        highlights.append({
            "text": arb_match.group(1),
            "category": "LegalTrap"
        })
    
    # Find class action waiver
    class_match = re.search(r'(class action waiver[^.]{0,30})', text, re.IGNORECASE)
    if class_match:
        highlights.append({
            "text": class_match.group(1),
            "category": "LegalTrap"
        })
    
    # Find rollover/renewal language
    rollover_match = re.search(r'((automatically renew|rollover|may be renewed)[^.]{0,40})', text, re.IGNORECASE)
    if rollover_match and features.get('has_rollover_or_renewal', 0):
        highlights.append({
            "text": rollover_match.group(1),
            "category": "DebtCycle"
        })
    
    # Find continuous debit language
    debit_match = re.search(r'((repeatedly debit|continuous.*authorization)[^.]{0,40})', text, re.IGNORECASE)
    if debit_match:
        highlights.append({
            "text": debit_match.group(1),
            "category": "PaymentAccess"
        })
    
    # Find employer contact
    employer_match = re.search(r'(contact.*employer[^.]{0,30})', text, re.IGNORECASE)
    if employer_match:
        highlights.append({
            "text": employer_match.group(1),
            "category": "Collection"
        })
    
    return highlights[:6]  # Limit to 6 highlights

def analyze_loan(text):
    """Complete analysis pipeline - returns API-ready response."""
    # Get hybrid score
    result = hybrid_score(text)
    
    # Generate reasons and highlights
    reasons = generate_reasons(result['features'])
    highlights = extract_highlights(text, result['features'])
    
    # Build response
    response = {
        "score": result['score'],
        "label": result['label'],
        "confidence": result['confidence'],
        "reasons": reasons,
        "highlights": highlights,
        "debug": {
            "rule_score": result['rule_score'],
            "ml_score": result['ml_score'],
            "ml_prob": result['ml_prob']
        }
    }
    
    return response

print("✓ Explanation and highlight functions defined")

✓ Explanation and highlight functions defined


## 10. End-to-End Testing

In [27]:
# Test 1: Predatory loan
predatory_sample = """PAYDAY LOAN AGREEMENT — PREDATORY

Loan Amount: $300
APR: 520%

Fees:
Service Fee: $25 per $100 borrowed
Renewal Fee: $50
Late Fee: $50

Repayment:
Term: 14 days

Debt Cycle:
Automatic renewal if unpaid. Renewal fees apply.

Collection:
Lender may contact employer for collection purposes.

Legal:
Binding arbitration + class action waiver + jury trial waiver.
"""

print("="*60)
print("TEST 1: PREDATORY LOAN")
print("="*60)
result1 = analyze_loan(predatory_sample)
print(json.dumps(result1, indent=2))

TEST 1: PREDATORY LOAN
{
  "score": 98,
  "label": "Predatory",
  "confidence": "High",
  "reasons": [
    "APR is 520%, which is extremely high and predatory.",
    "Loan includes rollover/renewal clauses that can trap borrowers in debt cycles.",
    "Mandatory arbitration + class action waiver severely limits your legal rights.",
    "Fees charged per $100 borrowed compound quickly on short-term loans.",
    "Very short repayment term (14 days or less) makes it difficult to repay without rolling over."
  ],
  "highlights": [
    {
      "text": "APR: 520%",
      "category": "ExcessiveCost"
    },
    {
      "text": "$25 per $100 borrowed\nRenewal Fee: $50\nLat",
      "category": "ExcessiveCost"
    },
    {
      "text": "Binding arbitration + class action waiver + jury trial waiver",
      "category": "LegalTrap"
    },
    {
      "text": "class action waiver + jury trial waiver",
      "category": "LegalTrap"
    },
    {
      "text": "contact employer for collection purposes"

c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [28]:
# Test 2: Safe loan
safe_sample = """STANDARD CREDIT AGREEMENT — SAFE

Loan Amount: $8,500
APR: 10%

Fees:
Origination Fee: 1.25%
Late Fee: $25 after 15 days

Repayment:
Term: 30 months
Payment Schedule: Monthly

Transparency:
APR disclosed. Fee schedule included.

Legal:
No confession of judgment. No class action waiver.

Authorization:
Borrower may revoke electronic payment authorization at any time.
"""

print("="*60)
print("TEST 2: SAFE LOAN")
print("="*60)
result2 = analyze_loan(safe_sample)
print(json.dumps(result2, indent=2))

TEST 2: SAFE LOAN
{
  "score": 0,
  "label": "Safe",
  "confidence": "High",
  "reasons": [
    "Loan includes clear APR and fee disclosures.",
    "Contract includes borrower-friendly language and rights."
  ],
  "highlights": [
    {
      "text": "class action waiver",
      "category": "LegalTrap"
    }
  ],
  "debug": {
    "rule_score": 0,
    "ml_score": 0,
    "ml_prob": 5.256910430611548e-58
  }
}


c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


## 11. Calibration Test (Full Dataset)

In [29]:
# Test on all samples to ensure calibration
print("Running calibration test on full dataset...\n")

safe_dir = Path('dataset/safe')
predatory_dir = Path('dataset/predatory')

safe_scores = []
predatory_scores = []

# Test safe loans
for file_path in safe_dir.glob('*.txt'):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
        result = analyze_loan(text)
        safe_scores.append(result['score'])

# Test predatory loans
for file_path in predatory_dir.glob('*.txt'):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
        result = analyze_loan(text)
        predatory_scores.append(result['score'])

print("="*60)
print("CALIBRATION RESULTS")
print("="*60)
print(f"\nSafe Loans (should be < 50):")
print(f"  Count: {len(safe_scores)}")
print(f"  Mean: {np.mean(safe_scores):.1f}")
print(f"  Min: {np.min(safe_scores):.0f}")
print(f"  Max: {np.max(safe_scores):.0f}")
print(f"  Correctly classified: {sum(s < 50 for s in safe_scores)}/{len(safe_scores)}")

print(f"\nPredatory Loans (should be > 50):")
print(f"  Count: {len(predatory_scores)}")
print(f"  Mean: {np.mean(predatory_scores):.1f}")
print(f"  Min: {np.min(predatory_scores):.0f}")
print(f"  Max: {np.max(predatory_scores):.0f}")
print(f"  Correctly classified: {sum(s > 50 for s in predatory_scores)}/{len(predatory_scores)}")

overall_accuracy = (sum(s < 50 for s in safe_scores) + sum(s > 50 for s in predatory_scores)) / (len(safe_scores) + len(predatory_scores))
print(f"\nOverall Calibration Accuracy: {overall_accuracy*100:.1f}%")

if overall_accuracy >= 0.95:
    print("\n✓ EXCELLENT calibration!")
elif overall_accuracy >= 0.85:
    print("\n✓ Good calibration")
else:
    print("\n⚠ May need adjustment")

Running calibration test on full dataset...



c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\

CALIBRATION RESULTS

Safe Loans (should be < 50):
  Count: 38
  Mean: 1.9
  Min: 0
  Max: 9
  Correctly classified: 38/38

Predatory Loans (should be > 50):
  Count: 32
  Mean: 69.5
  Min: 3
  Max: 99
  Correctly classified: 26/32

Overall Calibration Accuracy: 91.4%

✓ Good calibration


c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


## 12. Export Functions for Backend Integration

In [30]:
# Save key functions to a Python module for backend use
module_code = '''"""LoanShark AI - ML Inference Module

This module contains all functions needed for backend integration.
Import this in your Flask/FastAPI backend.
"""

# Copy all the functions from this notebook:
# - extract_features()
# - predict_ml()
# - calculate_rule_score()
# - hybrid_score()
# - generate_reasons()
# - extract_highlights()
# - analyze_loan()

# Usage in backend:
# from loanshark_inference import analyze_loan
# result = analyze_loan(loan_text)
# return jsonify(result)
'''

with open('loanshark_inference.py', 'w') as f:
    f.write(module_code)

print("✓ Template module created: loanshark_inference.py")
print("\nNext steps:")
print("1. Copy all function definitions from this notebook to loanshark_inference.py")
print("2. Import in your backend: from loanshark_inference import analyze_loan")
print("3. Use: result = analyze_loan(text)")
print("4. Return as JSON response")

✓ Template module created: loanshark_inference.py

Next steps:
1. Copy all function definitions from this notebook to loanshark_inference.py
2. Import in your backend: from loanshark_inference import analyze_loan
3. Use: result = analyze_loan(text)
4. Return as JSON response


## Summary

✓ Feature extraction working (30+ features)  
✓ Dataset loaded and processed  
✓ Models trained and evaluated  
✓ Best model selected and saved  
✓ Hybrid scoring implemented  
✓ Explanations and highlights working  
✓ End-to-end pipeline tested  
✓ Calibration verified  

**Ready for backend integration!**